In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
import pandas as pd

In [7]:
df = pd.read_csv("ordijegeren.csv", header=None)

In [8]:
df

,0
0,en
1,kvelende
2,varm
3,sommerdag
4,det
...,...
2139,øye
2140,på
2141,den
2142,hvite


In [30]:
text = ' '.join(df[0].tolist())

In [31]:
tokenizer = Tokenizer()

In [32]:
tokenizer.fit_on_texts([text])

In [33]:
tokenizer.word_index

{'og': 1,
 'det': 2,
 'er': 3,
 'en': 4,
 'jeg': 5,
 'du': 6,
 'i': 7,
 'på': 8,
 'til': 9,
 'å': 10,
 'som': 11,
 'med': 12,
 'ikke': 13,
 'har': 14,
 'seg': 15,
 'for': 16,
 'av': 17,
 'han': 18,
 'meg': 19,
 'jegor': 20,
 'om': 21,
 'at': 22,
 'men': 23,
 'et': 24,
 'den': 25,
 'deg': 26,
 'så': 27,
 'hun': 28,
 'pelageja': 29,
 'bare': 30,
 'ser': 31,
 'går': 32,
 'sier': 33,
 'vlasytsj': 34,
 'kan': 35,
 'jo': 36,
 'fra': 37,
 'noe': 38,
 'de': 39,
 'sin': 40,
 'ham': 41,
 'eller': 42,
 'var': 43,
 'hva': 44,
 'ut': 45,
 'ha': 46,
 'selv': 47,
 'aldri': 48,
 'helt': 49,
 'etter': 50,
 'hans': 51,
 'over': 52,
 'opp': 53,
 'vi': 54,
 'gjøre': 55,
 'får': 56,
 'også': 57,
 'blir': 58,
 'når': 59,
 'eneste': 60,
 'kunne': 61,
 'mann': 62,
 'mens': 63,
 'holder': 64,
 'lenge': 65,
 'siden': 66,
 'gang': 67,
 'se': 68,
 'fått': 69,
 'være': 70,
 'tar': 71,
 'skyggeluen': 72,
 'enn': 73,
 'farvel': 74,
 'alt': 75,
 'skulle': 76,
 'står': 77,
 'tier': 78,
 'hånden': 79,
 'blikket': 80,
 

In [37]:
tokenized_text = tokenizer.texts_to_sequences([text])[0]

In [38]:
input_sequences = []
for i in range(1, len(tokenized_text)):
    input_sequences.append(tokenized_text[:i+1])

In [40]:
max_len = max([len(x) for x in input_sequences])

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

In [42]:
x = padded_input_sequences[:, :-1]
y = padded_input_sequences[:, -1]

In [43]:
print(x.shape)
print(y.shape)

(2144, 2144)
(2144,)


In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [55]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=max_len-1))
model.add(LSTM(100))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

In [59]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 2144, 50)          35400     
                                                                 
 lstm_3 (LSTM)               (None, 100)               60400     
                                                                 
 dense_4 (Dense)             (None, 708)               71508     
                                                                 
Total params: 167,308
Trainable params: 167,308
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
67/67 [==============================] - 60s 883ms/step - loss: 6.1440 - accuracy: 0.0354
Epoch 2/100
67/67 [==============================] - 67s 995ms/step - loss: 5.7098 - accuracy: 0.0448
Epoch 3/100
67/67 [==============================] - 69s 1s/step - loss: 5.6529 - accuracy: 0.0462
Epoch 4/100
67/67 [==============================] - 72s 1s/step - loss: 5.5891 - accuracy: 0.0448
Epoch 5/100
67/67 [==============================] - 75s 1s/step - loss: 5.5164 - accuracy: 0.0448
Epoch 6/100
67/67 [==============================] - 76s 1s/step - loss: 5.4564 - accuracy: 0.0462
Epoch 7/100
67/67 [==============================] - 71s 1s/step - loss: 5.3922 - accuracy: 0.0513
Epoch 8/100
67/67 [==============================] - 66s 989ms/step - loss: 5.3713 - accuracy: 0.0597
Epoch 9/100
67/67 [==============================] - 65s 975ms/step - loss: 5.2737 - accuracy: 0.0616
Epoch 10/100
67/67 [==============================] - 66s 985ms/step - loss: 5.2014 - accuracy: 0

In [112]:
loss, accuracy = model.evaluate(x, y)
print(f"accuracy: {accuracy * 100:.2f}%")

67/67 [==============================] - 17s 248ms/step - loss: 0.1808 - accuracy: 0.9953
accuracy: 99.53%


In [113]:
import numpy as np

In [118]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted = np.random.choice(len(predicted_probs), p=predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [119]:
def get_random_seed():
    random_index = np.random.randint(0, len(text))
    return text[random_index]

In [120]:
next_words = 100
random_word = get_random_seed()
generated_text = generate_text(random_word, next_words, max_len)

In [121]:
print(f"text: {generated_text}")

text: e sommerdag det er en som virkelig av begynt det er som mann en eneste gang har det visst at du ser på meg med å drive med jakt eller driver omkring og helt han han og det da du bare bare går og så vilje i hytten stemme kalle på han og tar de han en annen kunstart da duger han senere hverken til embetsmann eller godseier du venne deg på ditt dumme kvinnfolk og til og sier til og når jeg så er sammen med du til å gjøre som hva slags menneske hva jeg greier meg det driver
